In [1]:
%matplotlib inline

"""
====================================================================
Create an RHI plot with reflectivity contour lines from an MDV file
====================================================================

An example which creates an RHI plot of velocity using a RadarDisplay object
and adding Reflectivity contours from the same MDV file.

"""
print __doc__

# Author: Cory Weber (cweber@anl.gov)
# License: BSD 3 clause
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from owslib.wms import WebMapService
import pyart
import numpy as np
import os





Create an RHI plot with reflectivity contour lines from an MDV file

An example which creates an RHI plot of velocity using a RadarDisplay object
and adding Reflectivity contours from the same MDV file.



## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119 


In [2]:
def plot_rhi(in_fn,out_fn):
    #Since the names in the HDF5 file are non-standard we use file_field_names = True
    #Since the names in the HDF5 file are non-standard we use file_field_names = True
    
    radar = pyart.aux_io.read_odim_h5(in_fn, file_field_names=True) 
    
    #rename fields
    radar.fields['DBZH']['standard_name'] = 'Reflectivity'
    radar.fields['DBZH']['units'] = 'dBZ'
    radar.fields['DBZH']['long_name'] = 'Radar Reflectivity Factor'
    radar.fields['VRADH']['standard_name'] = 'Velocity'
    radar.fields['VRADH']['units'] = 'm/s'
    radar.fields['VRADH']['long_name'] = 'Radial Velocity of Scatterers'

    #create gate filter
    gatefilter = pyart.correct.GateFilter(radar)
    gatefilter.exclude_below('DBZH', 20)
    
    #create radar display object
    plt.clf()
    display = pyart.graph.RadarMapDisplay(radar)
    
    #refl plot
    plt.subplot(1,2,1)
    #create basemap
    ref_m = Basemap(llcrnrlon=min_lon,
            llcrnrlat=min_lat,
            urcrnrlon=max_lon,
            urcrnrlat=max_lat, 
            projection='tmerc', 
            resolution = bm_res,
            epsg = 3857)
    #load background image
    im = plt.imread('background.png')
    ref_m.imshow(im,zorder = 0,origin='upper')
    #plot radar data
    display.plot_ppi_map('DBZH', sweep=sweep, resolution=bm_res,
                         vmin=refl_min, vmax=refl_max, mask_outside=False,
                         cmap=pyart.graph.cm.NWSRef,lat_lines=lal, lon_lines=lol,
                         gatefilter=gatefilter, basemap = ref_m, zorder = 1)
    #overlay mapping data
    im = plt.imread('overlay.png')
    ref_m.imshow(im,zorder = 2,origin='upper')
    #overlay countries
    display.basemap.drawcounties()
    
    #refl plot
    plt.subplot(1,2,2)
    #create basemap
    ref_m = Basemap(llcrnrlon=min_lon,
            llcrnrlat=min_lat,
            urcrnrlon=max_lon,
            urcrnrlat=max_lat, 
            projection='tmerc', 
            resolution = bm_res,
            epsg = 3857)
    #load background image
    im = plt.imread('background.png')
    ref_m.imshow(im,zorder = 0,origin='upper')
    #plot radar data
    display.plot_ppi_map('VRADH', sweep=sweep, resolution=bm_res,
                         vmin=vel_min, vmax=vel_max, mask_outside=False,
                         cmap=pyart.graph.cm.NWSVel,lat_lines=lal, lon_lines=lol,
                         gatefilter=gatefilter, basemap = ref_m, zorder = 1)
    #overlay mapping data
    im = plt.imread('overlay.png')
    ref_m.imshow(im,zorder = 2,origin='upper')    
    #overlay countries
    display.basemap.drawcounties()
    # save figure
    plt.savefig(out_fn, dpi=100)
    

In [3]:
def generate_layers(max_lat,min_lat,max_lon,min_lon):
    
    #generate map bounds
    lat_dif   = max_lat-min_lat
    lon_dif   = max_lon-min_lon
    map_x_sz  = int(500*lon_dif)
    map_y_sz  = int(500*lat_dif)

    #create overly map
    wms = WebMapService('http://services.ga.gov.au/site_7/services/Topographic_Base_Map_WM/MapServer/WMSServer?', version='1.1.1')
    img = wms.getmap(layers=['Roads_4','Populated_Places_6'],srs='EPSG:4326',bbox=(min_lon, min_lat, max_lon, max_lat),size=(map_x_sz, map_y_sz),format='image/png',transparent=True)
    out = open('overlay.png', 'wb')
    out.write(img.read())
    out.close()  

    #create background map
    wms = WebMapService('http://ows.terrestris.de/osm-gray/service?', version='1.1.1')
    img = wms.getmap(layers=['TOPO-WMS'],srs='EPSG:4326',bbox=(min_lon, min_lat, max_lon, max_lat),size=(map_x_sz, map_y_sz),format='image/png',transparent=True)
    out = open('background.png', 'wb')
    out.write(img.read())
    out.close()

In [4]:
#config vars
root_path    = '/run/media/meso/DATA/obs_data/uqxpol-20161108/odimh5/'
out_dir      = '/run/media/meso/DATA/obs_data/uqxpol-20161108/images_zoom/'
sweep        = 4
refl_min     = 20
refl_max     = 65
vel_min      = -20
vel_max      = 20
max_lat      = -27.25
min_lat      = -27.65
min_lon      = 152.35
max_lon      = 152.85
lat_grid     = 0.1
lon_grid     = 0.1
bm_res       = 'h' #l (low), i (intermediate), h (high), f (full)
lal          = np.arange(min_lat, max_lat, lat_grid)
lol          = np.arange(min_lon, max_lon, lon_grid)

#generate wms layers
generate_layers(max_lat,min_lat,max_lon,min_lon)

#generate images
fls    = os.listdir(root_path)
fls.sort()
i = 1
fig = plt.figure(figsize=[18,7])
for fl in fls:
    print('processing file ',i,' ',root_path+ fl)
    if (i % 2 == 0):
        plot_rhi(root_path+ fl,out_dir+fl+'.png')
    else:
        plot_rhi(root_path+ fl,out_dir+fl+'.png')
    i = i +1
    
plt.close()

('processing file ', 1, ' ', '/run/media/meso/DATA/obs_data/uqxpol-20161108/odimh5/99_20161108_040900.h5')
('processing file ', 2, ' ', '/run/media/meso/DATA/obs_data/uqxpol-20161108/odimh5/99_20161108_041200.h5')
('processing file ', 3, ' ', '/run/media/meso/DATA/obs_data/uqxpol-20161108/odimh5/99_20161108_041500.h5')
('processing file ', 4, ' ', '/run/media/meso/DATA/obs_data/uqxpol-20161108/odimh5/99_20161108_041800.h5')
('processing file ', 5, ' ', '/run/media/meso/DATA/obs_data/uqxpol-20161108/odimh5/99_20161108_042100.h5')
('processing file ', 6, ' ', '/run/media/meso/DATA/obs_data/uqxpol-20161108/odimh5/99_20161108_042400.h5')
('processing file ', 7, ' ', '/run/media/meso/DATA/obs_data/uqxpol-20161108/odimh5/99_20161108_042700.h5')
('processing file ', 8, ' ', '/run/media/meso/DATA/obs_data/uqxpol-20161108/odimh5/99_20161108_043000.h5')
('processing file ', 9, ' ', '/run/media/meso/DATA/obs_data/uqxpol-20161108/odimh5/99_20161108_043300.h5')
('processing file ', 10, ' ', '/run/m